In [1]:
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [40]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"

# header is not necessary here
response = requests.get(url, headers=headers)

ipo_raw = pd.read_html(response.text)

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_19720\597940344.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)


In [49]:
def get_ipo_data(target):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    }

    url = "https://stockanalysis.com/ipos/" + target + "/"

    # header is not necessary here
    response = requests.get(url, headers=headers)

    ipo_raw = pd.read_html(response.text)
    
    return ipo_raw[0]

In [53]:
df_ipo_filings = get_ipo_data('filings/')
df_ipo_filings.head()

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_19720\3814367568.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
1,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
2,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
3,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,"Apr 16, 2024",GGL,Games Global Limited,-,-


In [54]:
df_ipo_filings.dtypes

Filing Date       object
Symbol            object
Company Name      object
Price Range       object
Shares Offered    object
dtype: object

# Q1

In [55]:
df_ipo_filings['Filing Date'] = pd.to_datetime(df_ipo_filings['Filing Date'])
df_ipo_filings['Shares Offered'] = pd.to_numeric(df_ipo_filings['Shares Offered'], errors='coerce').fillna(0)

In [6]:
def average_price_range(row):
    if row == "-":
        return np.NaN
    elif '-' in row:
        prices = row.replace('$','').split(' - ')
        prices = [float(item) for item in prices]

        return sum(prices)/len(prices)
    elif '$' in row:
        return float(row.replace('$',''))
    else:
        return row

In [56]:
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0
2,2024-04-22,DRJT,Derun Group Inc,$5.00,0.0
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0
4,2024-04-16,GGL,Games Global Limited,-,0.0
...,...,...,...,...,...
323,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
324,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
325,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
326,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0


In [57]:
df_ipo_filings['avg_price'] = df_ipo_filings['Price Range'].apply(average_price_range).fillna(0)
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price
0,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0,0.00
2,2024-04-22,DRJT,Derun Group Inc,$5.00,0.0,5.00
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,10.00
4,2024-04-16,GGL,Games Global Limited,-,0.0,0.00
...,...,...,...,...,...,...
323,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00
324,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00
325,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00
326,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00


In [58]:
df_ipo_filings['shares_offered_value'] = df_ipo_filings['Shares Offered'] * df_ipo_filings['avg_price']
df_ipo_filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,shares_offered_value
0,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,0.0,0.00,0.0
2,2024-04-22,DRJT,Derun Group Inc,$5.00,0.0,5.00,0.0
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,10.00,250000000.0
4,2024-04-16,GGL,Games Global Limited,-,0.0,0.00,0.0
...,...,...,...,...,...,...,...
323,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.00,13500000.0
324,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.00,50000000.0
325,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.00,22500000.0
326,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.00,17200000.0


In [59]:
df_ipo_fri_2023 = df_ipo_filings[(df_ipo_filings['Filing Date'].dt.dayofweek == 4) & (df_ipo_filings['Filing Date'].dt.year == 2023)]
q1_ans = df_ipo_fri_2023.groupby('Filing Date').agg(totalsum=('shares_offered_value', 'sum')).reset_index()

In [60]:
q1_ans['totalsum_M'] = round(q1_ans['totalsum']/1e6)
q1_ans

,Filing Date,totalsum,totalsum_M
0,2023-01-13,16875000.0,17.0
1,2023-02-10,35000000.0,35.0
2,2023-02-24,62500000.0,62.0
3,2023-03-24,0.0,0.0
4,2023-03-31,37375000.0,37.0
5,2023-04-21,16000000.0,16.0
6,2023-04-28,11250000.0,11.0
7,2023-06-09,10000000.0,10.0
8,2023-06-23,9750000.0,10.0
9,2023-06-30,14075000.0,14.0


In [73]:
df_ipo_fri_2023

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,avg_price,shares_offered_value
52,2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000.0,4.00,4800000.0
53,2023-12-29,EPSM,Epsium Enterprise Limited,-,0.0,0.00,0.0
56,2023-12-22,LZMH,LZ Technology Holdings Limited,-,0.0,0.00,0.0
57,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,0.0,0.00,0.0
62,2023-12-15,GIT,Going International Holding Company Limited,-,0.0,0.00,0.0
64,2023-12-08,ENGS,Energys Group Limited,$4.00 - $6.00,2000000.0,5.00,10000000.0
65,2023-12-08,LNKS,Linkers Industries Limited,$4.00 - $6.00,2200000.0,5.00,11000000.0
85,2023-10-27,RAY,Raytech Holding Limited,$4.00 - $5.00,1500000.0,4.50,6750000.0
92,2023-10-13,ORIS,Oriental Rise Holdings Limited,$4.00,2000000.0,4.00,8000000.0
95,2023-10-06,QMMM,QMMM Holdings Limited,$4.00,2125000.0,4.00,8500000.0


# Q2

In [64]:
df_ipo_2023 = get_ipo_data("2023")
df_ipo_2024 = get_ipo_data("2024")

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_19720\3814367568.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)
C:\Users\sean.chang\AppData\Local\Temp\ipykernel_19720\3814367568.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_raw = pd.read_html(response.text)


In [65]:
df_ipo_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.24,-14.50%
1,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$35.00,9.38%
2,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$49.85,78.04%
3,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.34,1.79%
4,"Apr 23, 2024",NCI,Neo-Concept International Group Holdings Limited,$4.00,$1.45,-63.75%


In [66]:
df_ipo = pd.concat([df_ipo_2023, df_ipo_2024], axis=0)
df_ipo = df_ipo[df_ipo['Symbol'] != 'RYZB']

In [71]:
df_ipo[df_ipo['Symbol'] == 'IROH']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 27, 2023",IROH,Iron Horse Acquisitions Corp.,$10.00,$10.04,0.40%


In [115]:
ticket_list = df_ipo['Symbol'].to_list()

In [98]:
df_iroh = yf.download(tickers='IROH', period='max', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [101]:
for i in range(1,31,1):
    df_iroh['growth_future_' + str(i) + 'd'] = df_iroh['Adj Close'] / df_iroh['Adj Close'].shift(i)
    growth_columns = [f'growth_future_{i}d' for i in range(1, 31)]
    quantiles_75th = df_iroh[growth_columns].quantile(0.75)
    optimal_day = quantiles_75th.idxmax()
    highest_quantile = quantiles_75th.max()

In [110]:
optimal_day

'growth_future_30d'

In [117]:
def download_ipo_data(ticket_list):
    dataframes = {}

    for i in ticket_list:
        df = yf.download(tickers=i, period='max', interval='1d')

        if not df.empty:
            dataframes[f'df_{i}'] = df
        else:
            print(f'No data found for {i}')

    return dataframes

In [118]:
stock_data_dfs = download_ipo_data(ticket_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for PTHR


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

No data found for IBAC


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BKHA']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed

No data found for BKHA



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [127]:
q2_ans = {}
for symbol, df in stock_data_dfs.items():
    day_highQ = {}
    for i in range(1,31,1):
        df['growth_future_' + str(i) + 'd'] = df['Adj Close'] / df['Adj Close'].shift(i)

    growth_columns = [f'growth_future_{i}d' for i in range(1, 31)]
    quantiles_75th = df[growth_columns].quantile(0.75)
    optimal_day = quantiles_75th.idxmax()
    highest_quantile = quantiles_75th.max()
    ans[symbol] = {
        'optimal_day':optimal_day,
        'highest_quantile':highest_quantile
    }

In [126]:
q2_ans

{'df_IROH': {'optimal_day': 'growth_future_30d',
  'highest_quantile': 1.0061365043664132},
 'df_LGCB': {'optimal_day': 'growth_future_29d',
  'highest_quantile': 1.6717171267141753},
 'df_ZKH': {'optimal_day': 'growth_future_28d',
  'highest_quantile': 1.1325244413942699},
 'df_BAYA': {'optimal_day': 'growth_future_24d',
  'highest_quantile': 1.0079522866009685},
 'df_INHD': {'optimal_day': 'growth_future_3d',
  'highest_quantile': 1.032200200979665},
 'df_AFJK': {'optimal_day': 'growth_future_29d',
  'highest_quantile': 1.0081000269057614},
 'df_GSIW': {'optimal_day': 'growth_future_26d',
  'highest_quantile': 1.6917769784089596},
 'df_FEBO': {'optimal_day': 'growth_future_30d',
  'highest_quantile': 1.5060240906174438},
 'df_CLBR': {'optimal_day': 'growth_future_29d',
  'highest_quantile': 1.014882604070479},
 'df_ELAB': {'optimal_day': 'growth_future_5d',
  'highest_quantile': 1.040487624574973},
 'df_RR': {'optimal_day': 'growth_future_19d',
  'highest_quantile': 1.226044095794292